# Item importance evaluation
This notebook aims to find out the importance of each item in our dataset, so based on the infos we (might) find out, we may direct our efforts to an especific model...

In [1]:
import numpy as np
import pandas as pd
from utils import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime
from catboost import CatBoost, CatBoostRegressor, Pool, cv
plt.rcParams['figure.figsize'] = (45, 10.0) # set default size of plots

NUMBER_OF_LAGS = 4

sys.path.append("../../main/datasets/")
!ls  ../../main/datasets/

1.0v.zip


In [2]:
infos, items, orders = read_data("../../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


## Preparing our dataset

In [3]:
# Changing our time signatures, 
# adding our promotion feature 
# and aggregating our data by weeks...
process_time(orders)

In [4]:
df = dataset_builder(orders, items)

## Feature creation

In [5]:
aux = pd.DataFrame()
for i in range(13, 0, -1):
    accum = df.loc[df.group_backwards >= i].groupby(['itemID']).agg(orderSum_median=('orderSum','median'))
    accum['group_backwards'] = i
    aux = pd.concat([aux, accum])
aux.reset_index(inplace=True)
aux['orderSum_median'] = aux.groupby('itemID')['orderSum_median'].shift(1)

df = pd.merge(df, aux, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards'])